### 모델 학습용 코드 구현 및 실행

- 학습별 코드 분리 (구분선 사용 및 해당 모델 이름 작성)
- 학습된 파라미터는 ./parameters 에 .pth 형식으로 저장하여 사용

In [1]:
input_file_path = ['./data/2022Data_part1.csv', './data/2022Data_part2.csv']

### Colab 사용시 주석 제거

# !rm -rf SKN19_2ND_5TEAM
# !git clone https://github.com/SKNetworks-AI19-250818/SKN19_2ND_5TEAM.git
# %cd SKN19_2ND_5TEAM

# import sys
# sys.path.append('/content/SKN19_2ND_5TEAM')
# input_file_path = ['/content/SKN19_2ND_5TEAM/data/encoded_dataset.csv']

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import random_split, DataLoader, ConcatDataset
import torch.nn as nn
import torch.optim as optim

import modules.DataAnalysis as DataAnalysis
import modules.ModelAnalysis as ModelAnalysis
import modules.DataModify as DataModify
from modules.DataSelect import DataPreprocessing

import modules.Models as Models

In [3]:

print(torch.__version__)
print(torch.version.cuda)


2.8.0+cu126
12.6


In [4]:
# 랜덤 시드 고정 : 결과 비교용
Models.set_seed(42)

dp = DataPreprocessing()

# device 설정 (cuda 사용 가능 시 cuda 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset 로드
dataset = DataModify.CancerDataset(
    target_column='target_label',              # target column
    time_column='Survival months_bin_3m',      # Survival months
    file_paths=input_file_path,
    transform=dp.run                           # 기존에 정제가 완료된 데이터를 사용할 경우 None
)

sui_input_file_path = ['./data/Suicide.csv']
sui_dataset = DataModify.CancerDataset(
    target_column='target_label',              # target column
    time_column='Survival months_bin_3m',      # Survival months
    file_paths=sui_input_file_path,
    transform=dp.run                           # 기존에 정제가 완료된 데이터를 사용할 경우 None
)

Using device: cuda


d:\SKN_19\SKN19_2ND_5TEAM\modules\DataModify.py:446: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [5]:
# train set size 설정 및 분리
# 전체 길이
n = len(dataset)

# 비율 설정
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 각 세트 크기 계산
train_size = int(n * train_ratio)
val_size = int(n * val_ratio)
test_size = n - train_size - val_size  # 합이 정확히 맞도록 조정

# 분리 수행
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_dataset = ConcatDataset([train_dataset, sui_dataset])

batch_size = 64

# 데이터를 로드
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# 모델 초기화
input_dim = dataset.data.shape[1]   # input dimension : data의 feature의 개수
hidden_size = (128, 64)             # 1번째, 2번째 hidden layer의 size
time_bins = 91                      # 3개월 단위로 time을 split하여 각 구간으로 삼음 -> 270개월+ 는 하나로 취급
num_events = 4                      # 사건의 개수

# 모델 선언
model = Models.DeepHitSurvWithSEBlockCNN(input_dim, hidden_size, time_bins, num_events, dropout=.2).to(device)

# 손실함수 및 optimizer 선언
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [6]:
# 모델 학습
def train_epoch(model, loader, optimizer, device=device):
    # 모델을 train 모드로 설정
    model.train()
    # loss 변수 선언
    total_loss, total_lik, total_rank = 0, 0, 0

    # loader에서 불러온 데이터를 기반으로 학습
    for x, times, events in loader:
        x, times, events = x.to(device), times.to(device), events.to(device)


        optimizer.zero_grad()
        logits, pmf, cif = model(x)
        loss, L_lik, L_rank = Models.deephit_loss(pmf, cif, times, events)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_lik += L_lik.item() * x.size(0)
        total_rank += L_rank.item() * x.size(0)

    n = len(loader.dataset)
    return total_loss/n, total_lik/n, total_rank/n

# 모델 평가
def evaluate(model, loader, device=device):
    # 모델을 평가 모드로 설정
    model.eval()
    total_loss, total_lik, total_rank = 0, 0, 0
    
    with torch.no_grad():
        for x, times, events in loader:
            x, times, events = x.to(device), times.to(device), events.to(device)

            logits, pmf, cif = model(x)
            loss, L_lik, L_rank = Models.deephit_loss(pmf, cif, times, events)

            total_loss += loss.item() * x.size(0)
            total_lik += L_lik.item() * x.size(0)
            total_rank += L_rank.item() * x.size(0)

    n = len(loader.dataset)
    return total_loss/n, total_lik/n, total_rank/n

def get_cif_from_model(model, loader, device=device):
    model.eval()
    all_cif = []
    all_times = []
    all_events = []
    with torch.no_grad():
        for x, times, events in loader:
            x = x.to(device)
            logits, pmf, cif = model(x)
            all_cif.append(cif.cpu())
            all_times.append(times)
            all_events.append(events)
    all_cif = torch.cat(all_cif, dim=0)  # (num_samples, num_events, time_bins)
    all_times = torch.cat(all_times, dim=0)
    all_events = torch.cat(all_events, dim=0)
    return all_cif, all_times, all_events

In [7]:
n_epochs = 20
for epoch in range(1, n_epochs+1):
    train_loss, train_lik, train_rank = train_epoch(model, train_loader, optimizer)
    val_loss, val_lik, val_rank = evaluate(model, val_loader)

    print(f"[{epoch:03d}] "
          f"Train Loss={train_loss:.4f} (L={train_lik:.4f}, R={train_rank:.4f}) | "
          f"Val Loss={val_loss:.4f} (L={val_lik:.4f}, R={val_rank:.4f})")


[001] Train Loss=0.9121 (L=0.9027, R=0.0187) | Val Loss=0.7227 (L=0.7157, R=0.0138)
[002] Train Loss=0.7550 (L=0.7474, R=0.0153) | Val Loss=0.7049 (L=0.6992, R=0.0114)
[003] Train Loss=0.7414 (L=0.7340, R=0.0149) | Val Loss=0.6878 (L=0.6818, R=0.0120)
[004] Train Loss=0.7368 (L=0.7294, R=0.0148) | Val Loss=0.6971 (L=0.6910, R=0.0122)
[005] Train Loss=0.7319 (L=0.7245, R=0.0148) | Val Loss=0.6831 (L=0.6768, R=0.0127)
[006] Train Loss=0.7280 (L=0.7207, R=0.0146) | Val Loss=0.6846 (L=0.6778, R=0.0137)
[007] Train Loss=0.7263 (L=0.7190, R=0.0146) | Val Loss=0.6999 (L=0.6920, R=0.0158)
[008] Train Loss=0.7242 (L=0.7170, R=0.0144) | Val Loss=0.6802 (L=0.6739, R=0.0126)
[009] Train Loss=0.7227 (L=0.7155, R=0.0145) | Val Loss=0.6813 (L=0.6753, R=0.0120)
[010] Train Loss=0.7219 (L=0.7147, R=0.0144) | Val Loss=0.6773 (L=0.6710, R=0.0128)
[011] Train Loss=0.7200 (L=0.7128, R=0.0144) | Val Loss=0.6791 (L=0.6726, R=0.0130)
[012] Train Loss=0.7201 (L=0.7130, R=0.0143) | Val Loss=0.6739 (L=0.6673, R=

In [8]:
torch.save(model.state_dict(), "./data/parameters/deephit_model_feature_cnn.pth")

In [21]:
input_params_path = './data/parameters/deephit_model_without_feature_concat.pth'

input_dim = dataset.data.shape[1]   # input dimension : data의 feature의 개수
hidden_size = (128, 64)             # 1번째, 2번째 hidden layer의 size
time_bins = 91                     # 3개월 단위로 time을 split하여 각 구간으로 삼음 -> 최대 270개월 + 그 후
num_events = 4                      # 사건의 개수

# 모델 정의 (학습할 때 사용한 모델 클래스)
model = Models.DeepHitSurvWithSEBlock(input_dim, 
                    hidden_size, 
                    time_bins, 
                    num_events,
                    )  # 사건 수 맞게 설정
model.load_state_dict(torch.load(input_params_path, map_location=device))
model.to(device)
model.eval()  # 평가 모드

DeepHitSurvWithSEBlock(
  (se_block): Sequential(
    (0): Linear(in_features=17, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=17, bias=True)
    (3): Sigmoid()
  )
  (se_block_event): ModuleList(
    (0-3): 4 x Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=64, bias=True)
      (3): Sigmoid()
    )
  )
  (shared): Sequential(
    (0): Linear(in_features=17, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
  )
  (heads): ModuleList(
    (0-3): 4 x Linear(in_features=64, out_features=91, bias=True)
  )
)

In [22]:
# train set CIF 추출
cif_train, times_train, events_train = get_cif_from_model(model, train_loader)

In [23]:
def compute_risk_score_sigmoid(pmf, time_lambda=0.05, event_weights=None):
    """
    pmf: torch.Tensor, shape (B, E, T) - 사건별 시간 확률
    time_lambda: float, 지수 감쇠 계수 (시간대 가중치)
    event_weights: list or torch.Tensor, 길이 E, 사건별 가중치
    """
    B, E, T = pmf.shape
    device = pmf.device

    # 시간 가중치
    time_weights = torch.exp(-time_lambda * torch.arange(T, device=device))
    
    # 사건 가중치
    if event_weights is None:
        event_weights = torch.ones(E, device=device)
    else:
        event_weights = torch.tensor(event_weights, device=device, dtype=torch.float32)
    
    # 가중치 적용
    weighted_pmf = pmf * time_weights.view(1, 1, T)
    weighted_pmf = weighted_pmf * event_weights.view(1, E, 1)

    # 가중합 계산
    risk_score_raw = weighted_pmf.sum(dim=(1, 2))

    # 0 기준으로 offset 제거 → 음수도 나오게
    risk_score_raw = risk_score_raw - risk_score_raw.mean()

    # 시그모이드 + 0~100 스케일
    risk_score = torch.sigmoid(risk_score_raw) * 100

    return risk_score

In [24]:
# 1️⃣ 모델에서 PMF 추출
def get_pmf_from_model(model, loader, device=device):
    model.eval()
    all_pmf = []
    all_times = []
    all_events = []
    with torch.no_grad():
        for x, times, events in loader:
            x = x.to(device)
            logits, pmf, _ = model(x)  # CIF는 필요 없음

            pmf = pmf[:, :, :-1]  # (batch_size, num_events, time_bins-1)
            
            all_pmf.append(pmf.cpu())
            all_times.append(times)
            all_events.append(events)
    all_pmf = torch.cat(all_pmf, dim=0)  # (num_samples, num_events, time_bins)
    all_times = torch.cat(all_times, dim=0)
    all_events = torch.cat(all_events, dim=0)
    return all_pmf, all_times, all_events

# train set PMF 추출
pmf_train, times_train, events_train = get_pmf_from_model(model, train_loader)

In [25]:
# 2️⃣ 사건별 가중치 설정
event_weights = [2.0, 2.0, 1.0, 7.0]  # 예시

# 3️⃣ 위험 점수 계산 (시그모이드 + 0~100)
risk_scores = compute_risk_score_sigmoid(pmf_train, time_lambda=0.05, event_weights=event_weights).numpy()

# 4️⃣ 통계 확인
print("최대값:", np.max(risk_scores))
print("최소값:", np.min(risk_scores))
print("평균값:", np.mean(risk_scores))
print("앞 10개 값:", risk_scores[:10])

# 5️⃣ 사건별 통계
events_np = events_train.numpy()
unique_events = np.unique(events_np)

print("=== 라벨별 Risk Score 통계 ===")
for e in unique_events:
    mask = (events_np == e)
    scores_e = risk_scores[mask]
    if len(scores_e) == 0:
        continue
    print(f"\nEvent {e}:")
    print(f"  개수: {len(scores_e)}")
    print(f"  최대값: {np.max(scores_e):.4f}")
    print(f"  최소값: {np.min(scores_e):.4f}")
    print(f"  평균값: {np.mean(scores_e):.4f}")


최대값: 99.99998
최소값: 33.890053
평균값: 48.777374
앞 10개 값: [45.059364 58.211338 85.037155 68.17443  36.553883 39.81885  41.093353
 60.54232  33.995552 38.352715]
=== 라벨별 Risk Score 통계 ===

Event -1:
  개수: 366123
  최대값: 99.9809
  최소값: 33.8901
  평균값: 46.5169

Event 0:
  개수: 41868
  최대값: 99.9784
  최소값: 33.9178
  평균값: 67.2819

Event 1:
  개수: 5857
  최대값: 99.7696
  최소값: 33.9755
  평균값: 52.1473

Event 2:
  개수: 6164
  최대값: 99.9817
  최소값: 34.1014
  평균값: 50.9891

Event 3:
  개수: 2441
  최대값: 100.0000
  최소값: 34.0460
  평균값: 56.7662


In [29]:
# 사건별 마지막 CIF를 입력으로 사용
X_risk = cif_train[:, :, -2].numpy()  # (num_samples, num_events)
weights = [0.3, 0.3, 1, 3]

risk_target = np.zeros(X_risk.shape[0])
for i in range(len(events_train)):
    t_i = min(times_train[i], cif_train.shape[2]-2)  # 최대값 제한
    if events_train[i] >= 0:
        risk_target[i] = cif_train[i, events_train[i], t_i].item()
    else:
        risk_target[i] = cif_train[i, :, t_i].sum().item()  # 검열 처리

risk_model = Models.WeightedCoxRiskEstimator(num_events=X_risk.shape[1], weights=weights, device=device)
risk_model.fit(X_risk, times_train, events_train)

torch.save(risk_model.event_linears.state_dict(), "./data/parameters/risk_model_event_linears.pth")

In [30]:
risk_scores = risk_model.predict(X_risk)

print("최대값:", np.max(risk_scores))
print("최소값:", np.min(risk_scores))
print("평균값:", np.mean(risk_scores))
print("앞 10개 값:", risk_scores[:10])

최대값: 99.85215
최소값: 91.372025
평균값: 98.28919
앞 10개 값: [96.10712  99.62866  99.82557  97.37601  99.60299  99.56934  95.52262
 95.01497  92.355606 99.58003 ]


In [31]:

# tensor → numpy 변환
events_np = events_train.numpy()

# 사건 라벨 종류 (-1은 검열)
unique_events = np.unique(events_np)

print("=== 라벨별 Risk Score 통계 ===")
for e in unique_events:
    mask = (events_np == e)
    scores_e = risk_scores[mask]

    if len(scores_e) == 0:
        continue

    print(f"\nEvent {e}:")
    print(f"  개수: {len(scores_e)}")
    print(f"  최대값: {np.max(scores_e):.4f}")
    print(f"  최소값: {np.min(scores_e):.4f}")
    print(f"  평균값: {np.mean(scores_e):.4f}")



=== 라벨별 Risk Score 통계 ===

Event -1:
  개수: 366123
  최대값: 99.8521
  최소값: 91.3720
  평균값: 98.2895

Event 0:
  개수: 41868
  최대값: 99.8504
  최소값: 91.3720
  평균값: 98.2888

Event 1:
  개수: 5857
  최대값: 99.8485
  최소값: 91.3720
  평균값: 98.2733

Event 2:
  개수: 6164
  최대값: 99.8503
  최소값: 91.3721
  평균값: 98.2948

Event 3:
  개수: 2441
  최대값: 99.8482
  최소값: 91.3975
  평균값: 98.2797
